# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read [the following instructions](https://github.com/sdobnik/computational-semantics/blob/master/README.md) on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (wikipedia.txt.zip). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/)).  
When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [2]:
corpus_dir = '/Users/jacobcoles/Desktop/LT2213/computational-semantics-vt21/02-vector-semantics/assignment/wikipedia'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [5]:
numdims = 1000
svddim = 5

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(corpus_dir, numdims) #keep these words

wi = make_word_index(ktw) # word index
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) # sorted words

# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, svddim)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [6]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [2554 3774 3105  567  962  631  443  185  311  189  131   28   93  169
   81  125  151  408  194   90   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   66    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   24    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3   10    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16   88    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. We saved all three matrices [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (pretrained.zip). Download them and unpack them to a `pretrained` folder which should be a subfolder of the folder with this notebook:

In [7]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('./pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('./pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('./pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('./pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [8]:
# testing semantic space
print('house:', space_10k['house'])

house: [2554 3774 3105 ...    0    0    0]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [9]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 12
number of available word pairs to test: 774


Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [10]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [14]:
raw_similarities  = [cosine(w1, w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1, w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities  = [cosine(w1, w2, svdspace_10k) for w1, w2 in word_pairs]

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [15]:
rho, pval = stats.spearmanr(semantic_similarity, raw_similarities)
print("""Semantic Similarity vs. Raw Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

rho, pval = stats.spearmanr(semantic_similarity, ppmi_similarities)
print("""Semantic Similarity vs. PPMI Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

rho, pval = stats.spearmanr(semantic_similarity, svd_similarities)
print("""Semantic Similarity vs. SVD Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

Semantic Similarity vs. Raw Similarity:
rho     = 0.1522
p-value = 0.0000
Semantic Similarity vs. PPMI Similarity:
rho     = 0.4547
p-value = 0.0000
Semantic Similarity vs. SVD Similarity:
rho     = 0.4232
p-value = 0.0000


**Your answer should go here:**

The PPMI model correlated the similarity scores best with the real semantic similarity scores from the internet. However, the difference between the PPMI score and the SVD score appears to be marginal. This outcome is expected because the PPMI score takes into consideration how distinctive a word is in a given context, i.e. it minimises the importance of words that occur frrequently across all contexts. Additionally, SVD reduces noise/randomness in the data to help find a 'true' signal or information. 

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [16]:
rho, pval = stats.spearmanr(visual_similarity, raw_similarities)
print("""Visual Similarity vs. Raw Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

rho, pval = stats.spearmanr(visual_similarity, ppmi_similarities)
print("""Visual Similarity vs. PPMI Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

rho, pval = stats.spearmanr(visual_similarity, svd_similarities)
print("""Visual Similarity vs. SVD Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Raw Similarity:
rho     = 0.1212
p-value = 0.0007
Visual Similarity vs. PPMI Similarity:
rho     = 0.3838
p-value = 0.0000
Visual Similarity vs. SVD Similarity:
rho     = 0.3097
p-value = 0.0000


**Your answer should go here:**

The performance of the three matrices compared against the visual similarity ranks in the same order as the semantic similarity above however they perform lower across the board. The PPMI and SVD matrices performance drops off more sharply than the raw similarities, though they still outperform the raw similarity by a considerable margin. It is unsuprising that the performance overall is lower - the matrices were never taking visual similarity into account. It is a bit surprising that he raw similarity's performance does not drop off more than it did. This may indicate that the performance of the raw similarity was never particularly good to begin with.

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

**Your answer should go here:**

The above example is incorrect, but we can still apply this principle. We know in advance that the primary difference between a king and a queen (in terms of real entities) is gender. We can attempt to capture this semantic information in the vector space literally by taking the difference between the vectors of the two entities. 

Thus we hope that (king-queen) gives us the mapping from king to queen in the vector space, where our key assumption is that the mapping IS the gender difference. 

If we have a vector as the semantic representation of gender, which can map between similar types of entities/nouns which only differ in gender, then ((queen-king)+man) should approximate (woman), if the respective differences between king and queen, and men and women are their gender. 

Here is some helpful code that allows us to calculate such comparisons.

In [17]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [42]:
short = normalize(ppmispace_10k['short'])
light = normalize(ppmispace_10k['light'])
long = normalize(ppmispace_10k['long'])
heavy = normalize(ppmispace_10k['heavy'])

#light-heavy+long=short
find_similar_to(light - heavy + long, svdspace_10k)[:10]

ValueError: operands could not be broadcast together with shapes (10000,) (50,) 

**Your answer should go here:**

Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [26]:
up = normalize(svdspace_10k['up'])
high = normalize(svdspace_10k['high'])
down = normalize(svdspace_10k['down'])
low = normalize(svdspace_10k['low'])

#high-low+down
find_similar_to(high-low+down, svdspace_10k)[:10]

[('down', 0.9228344781988292),
 ('back', 0.8866341446782393),
 ('off', 0.8755815493034719),
 ('away', 0.8669123137105386),
 ('up', 0.8385302599781485),
 ('just', 0.8143869669353987),
 ('behind', 0.8114415466351721),
 ('front', 0.8048615445972288),
 ('left', 0.7870710896231837),
 ('home', 0.7829769169141294)]

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset is `mitchell_lapata_acl08.txt` included with this notebook.

Explanation of the task from Discord channel:

**What are we trying to achieve?**  
We want to create models, which can automatically capture differences in meaning of different phrases. These models should also be as good as we are (humans) in this task, right? Check example (1) from Mitchell/Lapata paper, it has two sentences which share the same words, but their meaning is completely different. We as humans can clearly see this difference, but how could a machine capture it?

It is intuitive that in order to get a meaning of a phrase, we might combine meaning of individual words in this phrase somehow, but how? First, we represent each word with the frequency vector based on the semantic space that we built before (e.g, frequency space, ppmi space, svd space). In other words, each word's meaning is represented by the number of times other words occur in the context, defined by window size. Now, we have a vector for `discussion` and a vector for `thrive`. How do we combine these vectors to get a single vector for the phrase `discussion thrive`?

Such methods of combining meaning vectors into a single item are called semantic composition methods (literally, because we compose semantic meaning of the phrase from its individuals). During the lecture, we tried different semantic composition methods: additive, multiplicative, combined.

Let's say we multiplied the vectors (went with the multiplicative method) and now we have one vector for our phrase.

Remember, we want to have a model that captures differences between the phrases; it means that if `discussion thrive` is our reference phrase, we need to get a different phrase to compare it with the reference one. How do we get this other phrase? Well, this other phrase can be either very similar to the reference or not similar at all, right? Let's say we decided to go with the second option and made/constructed a phrase `discussion digress`, which we know is very dissimilar to the reference phrase. We label this pair of phrases as having a low similarity, e.g., `low` in `hilo`. We can also create a different phrase (e.g., `discussion develop`) and use it as a high similarity phrase when paired with our reference phrase, right? This then would be labeled as `high` in `hilo`. This is what `hilo` in the dataset stands for: known information about how similar the reference phrase and the landmark phrase are.

Now, our main task is to automatically learn the similarities/differences between our reference and our landmark, right? We take the first pair: `discussion thrive` vs. `discussion digress`. We have a vector representation for each of these phrases. How do we compare them? We use cosine similarity to calculate a single score that would tell us about the similarity between these two vectors. The bigger the cosine, the more similar two vectors are. Cosine ranges from 0 to 1 (0 is very low similarity, 1 is very high similarity). Let's say, we get a cosine of 0.89, it means that our multiplicative model (remember, we decided to use multiplicative semantic composition method) thinks that these phrases are very similar (cosine is quite high). But wait a second, we know that these two phrases should be of low similarity, right? Because this is what the value in `hilo` tells us about this pair - `discussion thrive` and `discussion digress` are not similar to each other. Clearly, our multiplicative method fails to capture it. What can we do to improve our model? We can try a different composition method for getting a phrase vector from phrase's words: let's replace multiplication with addition. Or we can also use combined method. Let's say we used combined method and run cosine again; this time it tells that the cosine score is 0.45. Ok, this seems to be quite low, and it also agrees with our knowledge that these phrases are indeed not similar.

In other words, we need to evaluate different composition models (additive, multiplicative, combined, other methods from [2]) and analyse how well they perform. How do we analyse their performance? Because we have the ground-truth for comparison (`hilo` values), we know whether our phrases are actually similar or not. We want our cosine score to reflect this knowledge: if the score is high, but the groun-truth hilo is low, then we have a problem in the model - it did not learn things well, we need to replace the composition function.

_Long story short_: `hilo` is something that we compare our cosine to. `hilo` contains correct answers about similarity between phrases, and cosine should agree with this. If reference-landmark pair are `high` in `hilo`, then cosine should be high enough to reflecet that. If cosine is not high in this case, then we look at our model and change the composition function. We need to find the function, which give us cosines that are super close to the ground-truth known `hilo` values.

![title](./res.png)

Now, how do we compute something similar to the results from [2] (picture above):

In `High` and `Low` column you have mean cosine values.
These are calculated by basically averaging cosine scores for all pairs of phrases per model.
Rows introduce different models: `add` is additive, `multiply` is multiplicative, etc. (these are all described in the paper). `NonComp` is a baseline model, the most "stupid" one, it should be the worst. `UpperBound` is how humans performed in this task (they were asked to rate similarity between pairs of phrases, 1 is the lowest, 7 is the highest). Why these numbers are in a different scale, not from 0 to 1 like cosine, but from 1 to 7? Because they are not normalised, and authors explicitly said that they are interested in relative differences.

We need models which are closer to human ratings. `Add` model has 0.59 mean for `High` and 0.59 for `Low`, so it did not learn to differentiate between high similarity pairs and low similarity ones. This is a bad model then, we need a better one. `WeightAdd` seems to be doing better, the difference between `High` and `Low` is now 0.01, but it's also quite bad - the difference is not that obvious. The best models are `Multiply` and `Combined`, because their mean cosines for `High` and `Low` are quite different from each other. We can see that these two models gave higher cosines for high similarity pairs (0.42 and 0.38), while giving lower cosines for low similarity pairs (0.28 and 0.28). And this is a good result - it shows that these two composition functions are so far the best in (i) giving high cosine go highly similar pairs and low cosine to very dissimilar pairs, and (ii) keeping the distance (range) between high and low cosines quite large. However, we can't say how far/close they are when compared to human performance (UpperBound) since human scores are not normalised.

Still, we want to choose a model which is the closest to humans. This is why we are using the correlation test.

How do we perform the correlation test? We need to see how well *each model* correlated with human judgements. So for each model, we would have a vector of cosine values this models gives for each pair that we have. For example, let's say we have three pairs of phrases and our cosine values from additive model are the following ones: `[0.89, 0.40, 0.70]`. Now we need to get a vector of the same size, but for human scores. What do we have for human scores? We have multiple participants, which means that a single phrase can be evaluated by multiple participants. Let's say, the first phrase has scores from two participants (`6, 7`), so what we would do is that we would average it to have a single number (`(6 + 7) / 2 = 6.5`). With this, we can get a mean vector of human scores per item: `[6.5, 3, 6]`.

Now, we have two vectors and we can runk Spearman correlation on these vectors. This is what exactly what the third column in the Results table is showing (the correlation value). There is also a p-value (denoted right below the table).

**What is your ultimate task in this part of the assignment?**

(i) process the dataset and extract `reference - landmark` pairs; you can use the code from the lecture as something to start with. Try to keep information about human rating (`input`) and high/low similarity (`hilo`), because you will need it for correlation tests. Also, you might want to keep the information about participant id (will be useful for getting average numbers for correlation tests). Which format you should use to keep all this data? It's up to you, but a dictionary-like format could be a good choice.

(ii) build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair of phrases, and compute the composition of its vectors using each of these functions. Thus, you will have (at least) three compositional models that take each `noun - verb` phrase from the pair (these phrases can be either references or landmarks) and output a single vector, representing the meaning of this phrase. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper.

**Thoughts process behind calculating the correlation:**

Let's look at the example pair: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. Let's say we have 3 humans evaluating the similarity between these two phrases and we combine their scores into a single vector: `[5, 6, 5]`. We need to average them to get our human vector for correlation: `[5.3]`.

Our A model's output:  
`cosine(p1, p2) = 0.88`, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector `[5.3]` and model A output `[0.88]`. Next is to compute correlation between these two vectors. This should give you a correlation value and p-value for the model of choice and human ratings.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]). Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

The minimum to do in this task: compute correlations for at least ONE model and human ratings. However, this should not be hard to run it for any other model as well. For examples on how to interpret the results, look at Section 5 Results of the original paper.

### The rest is from the old description:
~~The paper states that they calculated correlations between each individual participant's judgeements and each model's predictions.  ~~

~~Let's say we have 3 models: simple additive (A), simple multiplicative (M), combined (C).  
From our task dataset, we also know that we have 20 participants.  
Now, for each participant in 20 participants we get all `verb - noun` pairs that these participated evaluated.  
For example:~~

In [ ]:
'''
participant_judgemenets_example = [
 'participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high'   
]
'''

In [ ]:
#participant_judgemenets_example

~~Let's look at the first pair that participant50 evaluated: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. The human gave the similarity score of 6 (very similar). Thus, human similarity judgment = [6].  ~~

~~Our A model's output:  
cosine(p1, p2) = 0.88, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  ~~

~~Therefore, we have human rating vector [6] and model A output [0.88]. Next is to compute correlation between these two vectors.~~

~~To get an overall score, simply average your correlation scores over all participants, since you are calculating correlation scores per participant.~~

~~Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]) where each element is a participant's judgement of a specific pair. Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.~~

~~The minimum to do in this task: compute correlations for 3 models mentioned above and human rating for AT LEAST one participant. Elaborate on how different the resulting correlation scores are depending on the model's composition function (additive, multiplicative, combined). For examples on how to interpret the results, look at Section 5 Results of the original paper.~~

In [ ]:
#your code should go here

**Any comments/thoughts should go here:**

# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.